 This tutorial uses the Python Google Earth
Engine API in conjunction with Google Colab, an online code editor that makes it easy to create
and share notebooks. Using Colab, we will demonstrate how to access and display satellite imagery
from Google Earth Engine. Then, we will perform some basic geoprocessing on the resulting
images. Finally, we will export the data to Google Drive.

**Before starting this tutorial, ensure that you have registered for Google Earth Engine and created a Cloud Project under this account. You will not be able to authenticate the notebook without doing so.**

## GEE Authentication and Initialization in Google Colab


In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()
#Input the Google Cloud Project ID here.
ee.Initialize(project="analog-vault-384203")

## Import Data from Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
path = "../content/drive/MyDrive/Earth Engine Colab/rhodeisland.geojson"
region = geemap.geojson_to_ee(path)

#Displays the GeoJSON as an EEObject through geemap.
map = geemap.Map()
map.centerObject(region, 9)
map.addLayer(region, {}, 'region')
map

Map(center=[41.5940076791208, -71.52472018824878], controls=(WidgetControl(options=['position', 'transparent_b…

## Retrieve Raster Data & Perform Operations


In [ ]:
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2');

In [ ]:
centroid = region.geometry().centroid(maxError=1)
spatialFiltered = l8.filterBounds(centroid)

In [ ]:
startDate = '2015-01-01';
endDate = '2015-12-31'
temporalFiltered = spatialFiltered.filterDate(startDate, endDate);

In [ ]:
temporalFiltered

## Cloud Masking

In [ ]:
def scaleL8sr(image : ee.Image) -> ee.Image:
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
  image = image.addBands(opticalBands, None, True)
  return image;

In [ ]:
def maskL8sr(image: ee.Image) -> ee.Image:
  qaMask = image.select('QA_PIXEL').bitwiseAnd(int(0b111111)).eq(0);
  saturationMask = image.select('QA_RADSAT').eq(0);
  return image.updateMask(qaMask).updateMask(saturationMask)

## Displaying Rasters


In [ ]:
def addNDVI(image : ee.Image) -> ee.Image:
  image = image.addBands(image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI'))
  return image

In [ ]:
l8 = temporalFiltered.map(scaleL8sr).map(maskL8sr)
visParams = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'max': 0.3};
map = geemap.Map()
map.centerObject(region, 9)
map.addLayer(l8, visParams, 'true-color composite');
map

Map(center=[41.5940076791208, -71.52472018824878], controls=(WidgetControl(options=['position', 'transparent_b…

## Perform Operations on Raster Data

In [ ]:
l8 = l8.map(addNDVI)
greenest = l8.qualityMosaic('NDVI')
map = geemap.Map()
map.centerObject(region, 9)
visParams = {'bands': ['NDVI'], 'min': -1, 'max': 1, 'palette': ['white', 'green']};
map.addLayer(greenest, visParams, 'Greenest pixel composite');
map

Map(center=[41.5940076791208, -71.52472018824878], controls=(WidgetControl(options=['position', 'transparent_b…

## Export Data to Drive

In [ ]:
task = ee.batch.Export.image.toDrive(
  image=greenest.select('NDVI'),
  crs='EPSG:4326',
  region=region.geometry(),
  description=f'NDVI_max_{startDate}_{endDate}',
  folder='Earth Engine Colab',
  scale=30);

task.start()

In [ ]:
task.status()

{'state': 'COMPLETED',
 'description': 'NDVI_max_2015-01-01_2015-12-31',
 'priority': 100,
 'creation_timestamp_ms': 1729872979962,
 'update_timestamp_ms': 1729873188776,
 'start_timestamp_ms': 1729872985836,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/18HHD0kONmKcaJNWI10rNGpOlZEVGnhMd'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 298.9755859375,
 'id': '6LR3EVEHH7TD4EBTYBOB6FNV',
 'name': 'projects/analog-vault-384203/operations/6LR3EVEHH7TD4EBTYBOB6FNV'}